In [1]:
import ee # Google Earth Engine
import geemap.eefolium as geemap # Visualização de dados
import geepy # PDI
import math # Cálculos matemáticos
import sys # Interação com o sistema

In [2]:
# Arquivo de parâmetros
config = geepy.params.configParams('input_params.json')

In [3]:
# Delimitação da área de estudo
hidrography = ee.FeatureCollection(config.params['auxiliaryData']['hidrography'])
buffer = hidrography.geometry().buffer(2000)
region = ee.FeatureCollection(buffer)

In [4]:
Map = geemap.Map(center=[-43.3290,-20.2594])
Map.centerObject(hidrography)
Map.addLayer(hidrography, {}, "hidrografia")
Map

In [5]:
# Coleções Landsat
l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['sr']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['sr']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['sr']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [6]:
print('Landsat 5')
print(config.params["imgCollection"]["lc5"]["bands"])
print(config.params["imgCollection"]["lc5"]["bandNames"])

print('Landsat 7')
print(config.params["imgCollection"]["lc7"]["bands"])
print(config.params["imgCollection"]["lc7"]["bandNames"])

print('Landsat 8')
print(config.params["imgCollection"]["lc8"]["bands"])
print(config.params["imgCollection"]["lc8"]["bandNames"])

Landsat 5
['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'pixel_qa']
['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal', 'qa']
Landsat 7
['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'pixel_qa']
['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal', 'qa']
Landsat 8
['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'pixel_qa']
['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal', 'qa']


In [7]:
# Período de processamento dos dados 
years2process = [x for x in range(
    config.params['years2process']['startYear'], 
    config.params['years2process']['endYear'])]

# Filtragem de nuvem e sombra, filtragem temporal
landsat = {}
for year in years2process:  
    start_d = str(year) + config.params['period']['dry']['start']
    end_d = str(year) + config.params['period']['dry']['end']

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) in (2002, 2011, 2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l7'
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) == 2018):
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, '2018-09-06').map(geepy.image.maskLandsatSR)
        satellite = 'l8'
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l8'
        
    # Remove bordas e faz a redução da imagem com base na mediana dos pixels
    medianEdgeRemoved = filtered.map(geepy.image.edgeRemoval).median()
    
    # Cálculo de índices
    medianEdgeRemoved = geepy.image.calcNDBI(medianEdgeRemoved)
    medianEdgeRemoved = geepy.image.calcNDVI(medianEdgeRemoved)
    medianEdgeRemoved = geepy.image.calcEVI(medianEdgeRemoved)
    medianEdgeRemoved = geepy.image.calcSAVI(medianEdgeRemoved)
    medianEdgeRemoved = geepy.image.calcNDWI_L(medianEdgeRemoved)
    medianEdgeRemoved = geepy.image.calcNDWI_WB(medianEdgeRemoved)
    medianEdgeRemoved = geepy.image.calcRatio(medianEdgeRemoved)
    
    medianEdgeRemoved = geepy.image.tassCapTransformation(medianEdgeRemoved, satellite)
        
    landsat[year] = medianEdgeRemoved.clip(region)
        
    sys.stdout.write("\rProcessing Landsat data: %s" % year)
    sys.stdout.flush()

Processing Landsat data: 2020

In [8]:
landsat_vis = {
    'min': 0,
    'max': 0.4,
    'bands': ['swir1', 'nir', 'red']
}

Map = geemap.Map(center=[-43.3290,-20.2594])
Map.centerObject(hidrography)
Map.addLayer(landsat[2010], landsat_vis, "Landsat 2010")
Map.addLayer(landsat[2015], landsat_vis, "Landsat 2015")
Map.addLayer(landsat[2020], landsat_vis, "Landsat 2020")

Map

In [9]:
# Calcula os valores máximos e mínimos para cada banda da colução de imagens
collectionFromImages = ee.ImageCollection.fromImages([landsat[year] for year in landsat])
maxValues = collectionFromImages.max()
minValues = collectionFromImages.min()

In [10]:
# Normalização
normalizedLandsat = {}
for year in years2process:
    numerator = landsat[year].subtract(minValues)
    denominator = maxValues.subtract(minValues)
    
    normalizedLandsat[year] = numerator.divide(denominator)

In [11]:
landsat_vis = {
    'min': 0.2,
    'max': 1.2,
    'bands': ['swir1', 'nir', 'red']
}

Map = geemap.Map(center=[-43.3290,-20.2594])
Map.centerObject(hidrography)
Map.addLayer(normalizedLandsat[2010], landsat_vis, "Landsat 2010")
Map.addLayer(normalizedLandsat[2015], landsat_vis, "Landsat 2015")
Map.addLayer(normalizedLandsat[2020], landsat_vis, "Landsat 2020")

Map

In [14]:
# MDE, slope, aspect, hillshade
srtm = ee.Image(config.params['auxiliaryData']['srtm'])

slope = ee.Terrain.slope(srtm)
aspect = ee.Terrain.aspect(srtm).divide(180).multiply(math.pi).sin()
hillshade = ee.Terrain.hillshade(srtm)

# Nighttime light data
# ntl30m = {}
# for i in years2process:
#     viirs = ee.Image(config.params['auxiliaryData']['VIIRS'][str(i)]).select('avg_rad').divide(100)
#     ntl30m[i] = viirs.resample('bilinear').reproject(
#         crs = viirs.projection().crs(),
#         scale = 30
#     )

In [12]:
landsat_vis = {
    'min': 0,
    'max': 0.4,
    'bands': ['swir1', 'nir', 'red']
}

Map = geemap.Map(center=[-43.3290,-20.2594])
Map.centerObject(hidrography)
Map.addLayer(landsat[2010], landsat_vis, "Landsat 2010")
Map.addLayer(landsat[2015], landsat_vis, "Landsat 2015")
Map.addLayer(landsat[2020], landsat_vis, "Landsat 2020")

Map

In [15]:
# Adicionando as camadas relacionadas ao relevo à pilha de bandas das imagens
for year in years2process:
    #normalizedLandsat[year] = geepy.image.img2Band(normalizedLandsat[year], ntl30m[year].clip(buffer), 'ntl')
    normalizedLandsat[year] = geepy.image.img2Band(normalizedLandsat[year], srtm.clip(buffer), 'srtm')
    normalizedLandsat[year] = geepy.image.img2Band(normalizedLandsat[year], slope.clip(buffer), 'slope')
    normalizedLandsat[year] = geepy.image.img2Band(normalizedLandsat[year], aspect.clip(buffer), 'aspect')
    normalizedLandsat[year] = geepy.image.img2Band(normalizedLandsat[year], hillshade.clip(buffer), 'hillshade')

In [16]:
Map = geemap.Map()
Map.centerObject(hidrography)

Map.addLayer(normalizedLandsat[2015].select(['srtm']), {'min': 400,'max': 900, 'palette': ['00A600','63C600','E6E600','E9BD3A','ECB176','EFC2B3','F2F2F2']}, "srtm")
Map.addLayer(normalizedLandsat[2015].select(['slope']), {'min': 0,'max': 100, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, "slope")
Map.addLayer(normalizedLandsat[2015].select(['aspect']), {'min': 0,'max': 1, 'palette': ['00A600','63C600','E6E600','E9BD3A','ECB176','EFC2B3','F2F2F2', 'D8D8D8']}, "aspect")
Map.addLayer(normalizedLandsat[2015].select(['hillshade']), {'min': 0,'max': 1000, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, "hillshade")

Map

In [17]:
# Obtem as amostras do Google Earth Engine
samples = ee.FeatureCollection(config.params['auxiliaryData']['samples'])

# Converte os poligonos para raster
raster_samples = geepy.feature.vec2rast(samples, 'class')

n = 1000 # nímero de amostras para cada clasee
classBand = 'class' # Nome do campo que contem os valores de clasee
cv = [1,2,3,4,5,6] # Valores de cada classe
cp = [n for i in range(len(cv))] #  Lista com o número de pontos para cada classe

# Gera os pontos aleatórios dentro de cada poligono
random_samples = geepy.image.randomSamples(samples, raster_samples, n, 369, classBand, cv, cp)
# 1 Lama ---------------------------------------------- #a3626e
# 2 Floresta ------------------------------------------ #149129
# 3 Capoeira ------------------------------------------ #11dc1b
# 4 Pastagem ------------------------------------------ #e5e160
# 5 Área construída ----------------------------------- #ff0000
# 6 Floresta plantada --------------------------------- #006419

In [18]:
Map = geemap.Map()
Map.centerObject(hidrography)
Map.addLayer(landsat[2015], landsat_vis, "Landsat 2015")
Map.addLayer(raster_samples.select('class'), {'min': 1,'max': 6, 'palette': ['a3626e','149129','11dc1b','e5e160','ff0000','006419']}, "Amostragem")

Map

In [19]:
# Separa as amostras de treinamento e validação
training = random_samples.filter(ee.Filter.gt('randCol', 0.2))
validation = random_samples.filter(ee.Filter.lt('randCol', 0.2))

# Obtem os dados de cada banda para cada ponto das amostras de treinamento
trained = geepy.image.trainingSamples(normalizedLandsat[2015], training)

In [20]:
Map = geemap.Map()
Map.centerObject(hidrography)
Map.addLayer(landsat[2015], landsat_vis, "Landsat 2015")
Map.addLayer(raster_samples.select('class'), {'min': 1,'max': 6, 'palette': ['a3626e','149129','11dc1b','e5e160','ff0000','006419']}, "Amostragem")
Map.addLayer(training, {}, "Treinamento")

Map

In [21]:
# Classificação das imagens usando o algoritmo Random Forest com base nas bandas selecionadas
bands = config.params['bandParams']
classification = {year: geepy.image.randomForest(normalizedLandsat[year], trained, bands, ntrees=10) for (year) in years2process}

In [22]:
bands

['blue',
 'green',
 'red',
 'nir',
 'swir1',
 'swir2',
 'ndbi',
 'ndvi',
 'evi',
 'savi',
 'ndwil',
 'ndwiwb',
 'ratio',
 'brightness',
 'greenness',
 'wetness',
 'srtm',
 'slope',
 'aspect',
 'hillshade']

In [23]:
Map = geemap.Map()
Map.centerObject(hidrography)
Map.addLayer(landsat[2016], {'min': 0, 'max': 0.2, 'bands': ['red', 'green', 'blue']}, "Landsat 2015")
Map.addLayer(classification[2015][0], {'min': 1,'max': 6, 'palette': ['a3626e','149129','11dc1b','e5e160','ff0000','006419']}, "Classificação 2015")

Map

In [13]:
# Cria as tarefas para enviar para o Google Earth Engine
tasks = {year: geepy.image.send2drive(classification[year][0], config.params['regionExtent'], 'classification' + str(year), 'classification', 30) for year in years2process}

In [14]:
# Executa as tarefas no Google Earth Engine
for i in tasks.keys(): [tasks[i].start()]